In [13]:
# for bart to modify

# globals (dev)
FOLDER_MODULES = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\modules'                    # change to your particular location
FOLDER_SHARED = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\shared'                      # change to your particular location
GRP_LYR_FILE = r'C:\Users\Lewis\Documents\GitHub\tenement-tools\arc\lyr\group_template.lyrx'  # change to your particular location

in_nc = r"C:\Users\Lewis\Desktop\bart_tute\phenolopy.nc"  # set location and file of your phenolopy output netcdf
out_folder = r'C:\Users\Lewis\Desktop\testing'  # set any random folder for output

In [14]:
# disable future warnings
import warnings
warnings.simplefilter(action='ignore', category=FutureWarning)

# imports
import os
import certifi
import arcpy

# safe imports
import os, sys       # arcgis comes with these
import datetime      # arcgis comes with this
import numpy as np   # arcgis comes with this
import pandas as pd  # arcgis comes with this

# risk imports (non-native to arcgis)
try:
    import xarray as xr  # not in arcgis
except:
    arcpy.AddError('Python library Xarray is not installed.')
    raise

# import tools
try:
    # shared folder
    sys.path.append(FOLDER_SHARED)
    import arc, satfetcher, tools  

    # module folder
    sys.path.append(FOLDER_MODULES)
    import phenolopy, cog   
except:
    arcpy.AddError('Could not find tenement tools python scripts (modules, shared).')
    raise

In [15]:
# run this function, which applys symbology to a layer (this could be the culprit)
def apply_cmap(aprx, lyr_name, cmap_name='Precipitation', cutoff_pct=0.5):
    """
    For current ArcGIS Project which runs this function,
    finds particular layer in table of contents (lyr_name),
    applies specific color map to it (cmap_name) with a
    percentage cutoff (cutoff_pct). Set cutoff_pct to 0
    for no thresholding (i.e., min-max colouring).
    
    Parameters
    -------------
    aprx : arcpy aprx object
        The currently selected ArcGIS Pro project.
    lyr_name : str
        Name of layer in current map to colourise.
    cmap_name : str
        Name of official ESRI colormaps to visualise 
        layer to.
    cutoff_pct : float
        Value to threshold layer colours to, 0 - 1.
        
    Returns
    ----------
    Original input layer file with new colourmap.
    """
    
    # get cmap if exists, precipitation if not
    try:
        cmap = aprx.listColorRamps(cmap_name)[0]
    except:
        print('Requested cmap does not exist. Using default.')
        cmap = aprx.listColorRamps('Precipitation')[0]
        
    # get active map and ensure it isnt empty
    m = aprx.activeMap
    if m is None:
        raise ValueError('No active map found. Please open a map first.')
    
    # get requested lyr
    lyr = m.listLayers(lyr_name)
    if len(lyr) != 1:
        raise ValueError('Requested layer not found.')
    elif not lyr[0].isRasterLayer:
        raise TypeError('Requested layer is not a raster type.')
        
    # prepare lyr and symbology objects
    lyr = lyr[0]
    sym = lyr.symbology
    
    # set symbology parameters
    sym.colorizer.stretchType = 'PercentClip'
    sym.colorizer.colorRamp = cmap
    sym.colorizer.minPercent = cutoff_pct
    sym.colorizer.maxPercent = cutoff_pct   
    sym.colorizer.invertColorRamp = False
    
    # set symbology of requested layer
    lyr.symbology = sym
    
    # return coloursed layer
    return lyr
    

In [16]:
# load the raw output xarray dataset
ds = xr.open_dataset(in_nc)

### Working

In [21]:
# # # # #
# add multi-dim raster to current map
in_add_result_to_map = True
if in_add_result_to_map:

    # notify and increment progess bar
    arcpy.SetProgressorLabel('Adding metrics to current ArcGIS map...')
    arcpy.SetProgressorPosition(16)

    # create output folder with dt
    dt = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    out_folder = os.path.join(os.path.dirname(in_nc), 'metrics' + '_' + dt)
    os.makedirs(out_folder)

    try:       
        # try to get current map, fail if doesnt exist
        aprx = arcpy.mp.ArcGISProject('CURRENT')
        m = aprx.activeMap

        # enable auto-add to map
        arcpy.env.addOutputsToMap = False
        
        # setup a group layer via template
        grp_lyr = arcpy.mp.LayerFile(GRP_LYR_FILE)
        grp = m.addLayer(grp_lyr)[0]
        grp.name = 'metrics'

        # loop each metric and export a seperate crf
        for var in list(ds.data_vars):
            
            print(var)

            # create current output crf file
            out_crf = os.path.join(out_folder, var + '.crf')

            # select current var from crf
            lyr = arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster=in_nc, 
                                                           out_multidimensional_raster_layer=out_crf,
                                                           variables=[var])

            # save crf into memory
            crf = arcpy.management.CopyRaster(in_raster=lyr, 
                                              out_rasterdataset=out_crf)

            # add to current map
            m.addDataFromPath(crf)

            # determine optimal cmap based on data type (or los)
            #if 'value' in var and 'los' not in var:
                #cmap, cutoff = 'Precipitation', 0.5
            #else:
                #cmap, cutoff = 'Temperature', 0.0

            # apply a colormap
            #lyr = apply_cmap(aprx=aprx, 
                             #lyr_name='{}.crf'.format(var),
                             #cmap_name=cmap,
                             #cutoff_pct=cutoff)

            # rename lyr, add to group, remove second instance
            #lyr.name = var     
            #m.addLayerToGroup(grp, lyr)
            #m.removeLayer(lyr)

    except Exception as e:
        print(e)
        arcpy.AddWarning('Could not visualise output.')


# # # # #
# clean up variables
arcpy.SetProgressorLabel('Finalising process...')
arcpy.SetProgressorPosition(17)

# close and del dataset
#ds.close()
#del ds

# notify user
arcpy.AddMessage('Generated Phenometrics successfully.')

pos_values
0.050906517
pos_times
15.0
vos_values
-0.13281909
vos_times
15.0
sos_values
-0.036052287
sos_times
0.0
eos_values
0.0
eos_times
0.0
roi_values
0.0
rod_values
0.0
lios_values
0.0
sios_values
0.0
liot_values
0.8423811
siot_values
0.11384606


### Testing

In [ ]:
# # # # #
# add multi-dim raster to current map
in_add_result_to_map = True
if in_add_result_to_map:

    # notify and increment progess bar
    arcpy.SetProgressorLabel('Adding metrics to current ArcGIS map...')
    arcpy.SetProgressorPosition(16)

    # create output folder with dt
    dt = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    out_folder = os.path.join(os.path.dirname(in_nc), 'metrics' + '_' + dt)
    os.makedirs(out_folder)

    try:       
        # try to get current map, fail if doesnt exist
        aprx = arcpy.mp.ArcGISProject('CURRENT')
        m = aprx.activeMap

        # enable auto-add to map
        arcpy.env.addOutputsToMap = False
        
        # setup a group layer via template
        grp_lyr = arcpy.mp.LayerFile(GRP_LYR_FILE)
        grp = m.addLayer(grp_lyr)[0]
        grp.name = 'metrics'

        # loop each metric and export a seperate crf
        for var in list(ds.data_vars):
            
            print(var)

            # create current output crf file
            out_crf = os.path.join(out_folder, var + '.crf')

            # select current var from crf
            lyr = arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster=in_nc, 
                                                           out_multidimensional_raster_layer=out_crf,
                                                           variables=[var])

            # save crf into memory
            crf = arcpy.management.CopyRaster(in_raster=lyr, 
                                              out_rasterdataset=out_crf)

            # add to current map
            m.addDataFromPath(crf)

            # determine optimal cmap based on data type (or los)
            #if 'value' in var and 'los' not in var:
                #cmap, cutoff = 'Precipitation', 0.5
            #else:
                #cmap, cutoff = 'Temperature', 0.0

            # apply a colormap
            #lyr = apply_cmap(aprx=aprx, 
                             #lyr_name='{}.crf'.format(var),
                             #cmap_name=cmap,
                             #cutoff_pct=cutoff)

            # rename lyr, add to group, remove second instance
            #lyr.name = var     
            #m.addLayerToGroup(grp, lyr)
            #m.removeLayer(lyr)

    except Exception as e:
        print(e)
        arcpy.AddWarning('Could not visualise output.')


# # # # #
# clean up variables
#arcpy.SetProgressorLabel('Finalising process...')
#arcpy.SetProgressorPosition(17)

# close and del dataset
#ds.close()
#del ds

# notify user
#arcpy.AddMessage('Generated Phenometrics successfully.')

In [130]:
# # # # #
# add multi-dim raster to current map
in_add_result_to_map = True
if in_add_result_to_map:

    # notify and increment progess bar
    arcpy.SetProgressorLabel('Adding metrics to current ArcGIS map...')
    arcpy.SetProgressorPosition(16)

    # create output folder with dt
    dt = datetime.datetime.now().strftime("%d%m%Y%H%M%S")
    out_folder = os.path.join(os.path.dirname(in_nc), 'metrics' + '_' + dt)
    os.makedirs(out_folder)

    try:       
        # try to get current map, fail if doesnt exist
        aprx = arcpy.mp.ArcGISProject('CURRENT')
        m = aprx.activeMap

        # enable auto-add to map
        arcpy.env.addOutputsToMap = False
        
        # setup a group layer via template
        #grp_lyr = arcpy.mp.LayerFile(GRP_LYR_FILE)
        #grp = m.addLayer(grp_lyr)[0]
        #grp.name = 'metrics'

        # loop each metric and export a seperate crf
        tifs = {}
        for var in list(ds.data_vars):

            # create temp tif from netcdf var in memory
            out_temp = os.path.join(out_folder, 'temp' + '_' + var + '.tif')
            lyr = arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster=in_nc, 
                                                           out_multidimensional_raster_layer=out_temp,
                                                           variables=[var])

            # save in memory tif to local 
            out_tif = os.path.join(out_folder, var + '.tif')
            arcpy.management.CopyRaster(in_raster=lyr, out_rasterdataset=out_tif)
                        
            # append path to list
            tifs.update({var: out_tif})

            # add to current map
            #m.addDataFromPath(crf)

            # determine optimal cmap based on data type (or los)
            #if 'value' in var and 'los' not in var:
                #cmap, cutoff = 'Precipitation', 0.5
            #else:
                #cmap, cutoff = 'Temperature', 0.0

            # apply a colormap
            #lyr = apply_cmap(aprx=aprx, 
                             #lyr_name='{}.crf'.format(var),
                             #cmap_name=cmap,
                             #cutoff_pct=cutoff)

            # rename lyr, add to group, remove second instance
            #lyr.name = var     
            #m.addLayerToGroup(grp, lyr)
            #m.removeLayer(lyr)

    except Exception as e:
        print(e)
        arcpy.AddWarning('Could not visualise output.')


# # # # #
# clean up variables
#arcpy.SetProgressorLabel('Finalising process...')
#arcpy.SetProgressorPosition(17)

# close and del dataset
#ds.close()
#del ds

# notify user
#arcpy.AddMessage('Generated Phenometrics successfully.')

In [147]:
for var, tif in tifs.items():
    
    # enable auto-add to map
    arcpy.env.addOutputsToMap = False
   
    sym = m.addDataFromPath(tif)
    
    # determine optimal cmap based on data type (or los)
    if 'value' in var and 'los' not in var:
        cmap, cutoff = 'Precipitation', 0.5
    else:
        cmap, cutoff = 'Temperature', 0.0

    # apply a colormap
    #sym = apply_cmap(aprx=aprx, 
                     #lyr_name='{}.tif'.format(var),
                     #cmap_name=cmap,
                     #cutoff_pct=cutoff)

    # rename lyr, add to group, remove second instance
    #lyr.name = var     
    #m.addLayerToGroup(grp, sym)
    #m.removeLayer(lyr)
    
    raise

RuntimeError: No active exception to reraise

In [149]:
help(m.moveLayer)

Help on method moveLayer in module arcpy._mp:

moveLayer(reference_layer, move_layer, insert_position='BEFORE') method of arcpy._mp.Map instance
    Map.moveLayer(reference_layer, move_layer, {insert_position})
    
    Provides the ability to move a layer or group layer in a map to  a
    specific location in the layer stack.
    
      reference_layer(Layer):
    A Layer object representing an existing layer that determines the
    location where the new layer will be moved.
    
      move_layer(Layer):
    A reference to a Layer object representing the layer to be moved.
    
      insert_position{String}:
    A constant that determines the placement of the moved layer relative
    to the reference layer.
    
     * AFTER: Moves the  layer after or below the reference layer.
    
     * BEFORE: Moves the layer before or above the reference layer.



In [159]:
out_tif

'C:\\Users\\Lewis\\Desktop\\bart_tute\\metrics_10122021232956\\siot_values.tif'

In [163]:
lyr = arcpy.md.MakeMultidimensionalRasterLayer(in_multidimensional_raster=in_nc, 
                                               out_multidimensional_raster_layer=out_temp,
                                               variables=['pos_values', 'pos_times', 'vos_values', 'vos_times'],
                                              dimensionless='NO_DIMENSIONS')

arcpy.management.CopyRaster(in_raster=lyr, out_rasterdataset=r'C:\\Users\\Lewis\\Desktop\\bart_tute\\metrics_10122021232956\\metrics.crf')

<Result 'C:\\\\Users\\\\Lewis\\\\Desktop\\\\bart_tute\\\\metrics_10122021232956\\metrics.crf'>